In [1]:
import os
import sys
import pickle
import psutil
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split

# Define data path
DATA_PATH = "data/"
CHECKPOINT_PATH = "models/"

In [2]:
pids = pickle.load(open(os.path.join(DATA_PATH,'pids.pkl'), 'rb'))
vids = pickle.load(open(os.path.join(DATA_PATH,'vids.pkl'), 'rb'))
targs = pickle.load(open(os.path.join(DATA_PATH,'targets.pkl'), 'rb'))
prob_targs = pickle.load(open(os.path.join(DATA_PATH, 'prob_targets_allvisits.pkl'), 'rb'))
seqs = pickle.load(open(os.path.join(DATA_PATH,'seqs.pkl'), 'rb'))
diags = pickle.load(open(os.path.join(DATA_PATH,'diags.pkl'), 'rb'))
codes = pickle.load(open(os.path.join(DATA_PATH,'icd9.pkl'), 'rb'))
categories = pickle.load(open(os.path.join(DATA_PATH,'categories.pkl'), 'rb'))
sub_categories = pickle.load(open(os.path.join(DATA_PATH,'subcategories.pkl'), 'rb'))
assert len(pids) == len(vids) == len(targs) == len(seqs)

In [3]:
class CustomDataset(Dataset):
    
    def __init__(self, seqs, targets):

        self.x = seqs
        self.y = targets
    
    def __len__(self):
        
        return(len(self.x))
    
    def __getitem__(self, index):
        
        return (self.x[index], self.y[index])

In [4]:
# dataset = CustomDataset(seqs, targs)
dataset = CustomDataset(seqs, prob_targs)

In [5]:
def collate_fn(data):
    """
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
        x: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.long
        x_masks: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.bool
        rev_x: same as x but in reversed time.
        rev_masks: same as x_masks but in reversed time.
        y: a tensor of shape (# patients, max # diagnosis categories) of type torch.float
        y_masks: a tensor of shape (# patients, max # diagnosis categories) of type torch.bool
    """
#     sequences, targets = zip(*data)

#     num_patients = len(sequences)
#     num_visits = [len(patient) for patient in sequences]
#     num_codes = [len(visit) for patient in sequences for visit in patient]
#     batch_num_categories = [len(visit) for patient in targets for visit in patient]
    
#     max_num_visits = max(num_visits)
#     max_num_codes = max(num_codes)
#     max_num_categories = max(batch_num_categories)
    
#     x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
#     y = torch.zeros((num_patients, max_num_categories), dtype=torch.long)
#     rev_x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
#     x_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
#     rev_x_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
#     y_masks = torch.zeros((num_patients, max_num_categories), dtype=torch.bool)

#     for i_patient, patient in enumerate(sequences):   
#         for j_visit, visit in enumerate(patient[:-1]):
#             for k_code, code in enumerate(visit):
#                 x[i_patient, j_visit, k_code] = code
#                 x_masks[i_patient, j_visit, k_code] = 1
#                 if j_visit == len(patient) - 2 and k_code == len(visit) - 1:
#                     rev_visit = x_masks[i_patient].any(dim=1)
#                     rev_x[i_patient, rev_visit] = x[i_patient, rev_visit].flip(0)
#                     rev_x_masks[i_patient, rev_visit] = x_masks[i_patient, rev_visit].flip(0)
  
#     for i_patient, patient in enumerate(targets):   
#         for visit in patient[-1:]:
#             for k_code, code in enumerate(visit):
#                 y[i_patient, k_code] = code
#                 y_masks[i_patient, k_code] = 1
    
#     return x, x_masks, rev_x, rev_x_masks, y, y_masks

    sequences, targets = zip(*data)

    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]
    batch_num_categories = [len(visit) for patient in targets for visit in patient]
    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)
#     max_num_categories = max(batch_num_categories)
    max_num_categories = len(targets[0][0])
    
    x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    rev_x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    x_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    rev_x_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
#     y = torch.zeros((num_patients, max_num_categories), dtype=torch.long)
    y = torch.zeros((num_patients, max_num_categories), dtype=torch.float)
    y_masks = torch.zeros((num_patients, max_num_categories), dtype=torch.bool)
    for i_patient, patient in enumerate(sequences):   
        for j_visit, visit in enumerate(patient[:-1]):
            for k_code, code in enumerate(visit):
                x[i_patient, j_visit, k_code] = code
                x_masks[i_patient, j_visit, k_code] = 1
                if j_visit == len(patient) - 2 and k_code == len(visit) - 1:
                    rev_visit = x_masks[i_patient].any(dim=1)
                    rev_x[i_patient, rev_visit] = x[i_patient, rev_visit].flip(0)
                    rev_x_masks[i_patient, rev_visit] = x_masks[i_patient, rev_visit].flip(0)

#     for i_patient, patient in enumerate(targets):
# #         import pdb; pdb.set_trace()
#         last_visit = patient[-1]
#         y[i_patient,:len(last_visit)] = torch.LongTensor(last_visit)
#         y_masks[i_patient,:len(last_visit)] = torch.BoolTensor(np.ones(len(last_visit)))
    for i_patient, patient in enumerate(targets):
#         import pdb; pdb.set_trace()
        last_visit = patient[-1]
        y[i_patient] = torch.FloatTensor(last_visit)
        y_masks[i_patient] = torch.BoolTensor(np.ones(max_num_categories))
    
    return x, x_masks, rev_x, rev_x_masks, y, y_masks

In [6]:
train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)
val_split = int(len(dataset)*0.10)

In [7]:
train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)

lengths = [train_split, test_split, len(dataset) - (train_split + test_split)]
train_dataset, test_dataset, val_dataset = random_split(dataset, lengths)

print("Length of train dataset:", len(train_dataset))
print("Length of test dataset:", len(test_dataset))
print("Length of val dataset:", len(val_dataset))

Length of train dataset: 6561
Length of test dataset: 1312
Length of val dataset: 875


In [8]:
def load_data(train_dataset, test_dataset, val_dataset, collate_fn):
    '''
    Arguments:
        train dataset: train dataset of type `CustomDataset`
        test dataset: test dataset of type `CustomDataset`
        val dataset: validation dataset of type `CustomDataset`
        collate_fn: collate function
        
    Outputs:
        train_loader, test_loader, val_loader: train, test and validation dataloaders
    '''
    batch_size = 100
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               collate_fn=collate_fn,
                                               shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset,
                                           batch_size=batch_size,
                                           collate_fn=collate_fn,
                                           shuffle=False)
    val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=batch_size,
                                             collate_fn=collate_fn,
                                             shuffle=False)
    
    return train_loader, test_loader, val_loader


train_loader, test_loader, val_loader = load_data(train_dataset, test_dataset, val_dataset, collate_fn)

In [9]:
def sum_embeddings_with_mask(x, masks):
    """
    Arguments:
        x: the embeddings of diagnosis sequence of shape (batch_size, # visits, # diagnosis codes, embedding_dim)
        masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

    Outputs:
        sum_embeddings: the sum of embeddings of shape (batch_size, # visits, embedding_dim)
    """
    x[~masks] = 0
    return x.sum(2)

In [10]:
def attention_sum(alpha, beta, rev_v, rev_masks):
    """
    Arguments:
        alpha: the alpha attention weights of shape (batch_size, # visits, 1)
        beta: the beta attention weights of shape (batch_size, # visits, hidden_dim)
        rev_v: the visit embeddings in reversed time of shape (batch_size, # visits, embedding_dim)
        rev_masks: the padding masks in reversed time of shape (batch_size, # visits, # diagnosis codes)

    Outputs:
        c: the context vector of shape (batch_size, hidden_dim)
    """
    rev_masks = rev_masks[:,:].any(dim=2)
    rev_v_masked = rev_v.clone()
    rev_v_masked[~rev_masks] = 0
    weights = alpha * beta
    c = torch.sum(weights * rev_v_masked, dim=1)
    return c

In [11]:
class AlphaAttention(torch.nn.Module):

    def __init__(self, hidden_dim):
        super().__init__()
        """
        Arguments:
            hidden_dim: the hidden dimension
        """
        
        self.a_att = nn.Linear(hidden_dim, 1)

    def forward(self, g):
        """
        Arguments:
            g: the output tensor from RNN-alpha of shape (batch_size, # visits, hidden_dim) 
        
        Outputs:
            alpha: the corresponding attention weights of shape (batch_size, # visits, 1)
        """
        m = nn.Softmax(dim=1)
        return m(self.a_att(g))

In [12]:
class BetaAttention(torch.nn.Module):

    def __init__(self, hidden_dim):
        super().__init__()
        """
        Arguments:
            hidden_dim: the hidden dimension
        """
        
        self.b_att = nn.Linear(hidden_dim, hidden_dim)


    def forward(self, h):
        """
        Arguments:
            h: the output tensor from RNN-beta of shape (batch_size, # visits, hidden_dim) 
        
        Outputs:
            beta: the corresponding attention weights of shape (batch_size, # visits, hidden_dim)
        """
        return torch.tanh(self.b_att(h))

In [13]:
class BaselineRETAIN(nn.Module):
    
    def __init__(self, num_codes, num_categories):
        super().__init__()
        """
        Arguments:
            num_codes: total number of diagnosis codes
            num_categories: total number of diagnosis categories to predict
        """
        self.embedding = nn.Embedding(num_codes, embedding_dim=180)
        self.rnn_a = nn.GRU(180, hidden_size=128, batch_first=True)
        self.rnn_b = nn.GRU(180, hidden_size=128, batch_first=True)
        self.att_a = AlphaAttention(128)
        self.att_b = AlphaAttention(128)
        self.fc = nn.Linear(180, num_categories)
    
    def forward(self, x, masks, rev_x, rev_masks):
        """
        Arguments:
            x: the diagnosis sequence of shape (batch_size, # visits, # diagnosis codes)
            masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)
            rev_x: the diagnosis sequence of shape (batch_size, # visits, # diagnosis codes)
            rev_masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

        Outputs:
            logits: logits of shape (batch_size, # categories)
        """
        rev_x = self.embedding(rev_x)
        rev_x = sum_embeddings_with_mask(rev_x, rev_masks)
        g, _ = self.rnn_a(rev_x)
        h, _ = self.rnn_b(rev_x)
        alpha = self.att_a(g)
        beta = self.att_b(h)
        c = attention_sum(alpha, beta, rev_x, rev_masks)
        logits = self.fc(c)
        return logits

# load the model here
baseline_retain = BaselineRETAIN(num_codes = len(codes), num_categories=len(sub_categories))
baseline_retain

BaselineRETAIN(
  (embedding): Embedding(4903, 180)
  (rnn_a): GRU(180, 128, batch_first=True)
  (rnn_b): GRU(180, 128, batch_first=True)
  (att_a): AlphaAttention(
    (a_att): Linear(in_features=128, out_features=1, bias=True)
  )
  (att_b): AlphaAttention(
    (a_att): Linear(in_features=128, out_features=1, bias=True)
  )
  (fc): Linear(in_features=180, out_features=184, bias=True)
)

In [14]:
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(naive_rnn.parameters(), lr=0.001)
optimizer = torch.optim.Adadelta(baseline_retain.parameters(), weight_decay=0.001)

In [15]:
def train(model, train_loader, test_loader, n_epochs):
    """
    Arguments:
        model: the BaselineRETAIN model
        train_loader: training dataloder
        test_loader: validation dataloader
        n_epochs: total number of epochs
    """
#     max_cpu, max_ram = print_cpu_usage()
#     for epoch in range(n_epochs):
#         model.train()
#         train_loss = 0
#         for x, x_masks, rev_x, rev_x_masks, y, y_masks in train_loader:
#             y_hat = model(x, x_masks, rev_x, rev_x_masks)
#             y_mh = indices_to_multihot(y, y_masks, y_hat)
#             loss = criterion(y_hat, y_mh)
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             train_loss += loss.item()
#         train_loss = train_loss / len(train_loader)
#         cpu, ram = print_cpu_usage()
#         max_cpu = cpu if cpu > max_cpu else max_cpu
#         max_ram = ram if ram > max_ram else max_ram
#         print(f'Epoch: {epoch+1} \t Training Loss: {train_loss:.6f}')
#         for k in range(5, 31, 5):
#             precision_k, accuracy_k = eval_model(model, test_loader, k=k)
#             print(f'Epoch: {epoch+1} \t Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')
#     final_cpu, final_ram = print_cpu_usage()
#     print(f"Max CPU usage: {max_cpu:.3f}\tMax RAM % usage: {max_ram}")
    
    max_cpu, max_ram = print_cpu_usage()
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x, x_masks, rev_x, rev_x_masks, y, y_masks in train_loader:
            y_hat = model(x, x_masks, rev_x, rev_x_masks)                  
#             y_mh = indices_to_multihot(y, y_masks, y_hat)
            loss = criterion(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        cpu, ram = print_cpu_usage()
        max_cpu = cpu if cpu > max_cpu else max_cpu
        max_ram = ram if ram > max_ram else max_ram
        print(f'Epoch: {epoch+1} \t Training Loss: {train_loss:.6f}')
        for k in range(5, 31, 5):
            precision_k, accuracy_k = eval_model(model, val_loader, k=k)
            print(f'Epoch: {epoch+1} \t Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')
    final_cpu, final_ram = print_cpu_usage()
    print(f"Max CPU usage: {max_cpu:.3f}\tMax RAM % usage: {max_ram}")

In [16]:
def eval_model(model, test_loader, k=15, n=-1):
    """
    Arguments:
        model: the BaselineRETAIN model
        test_loader: validation dataloader
        k: value for top k predictions
        n: num of records to evaluate in the batch, value -1 evaulates all records
        
    Outputs:
        precision_k: visit-level precison@k
        accuracy_k: code-level accuracy@k
    """
#     y_pred = torch.LongTensor()
#     y_true = torch.LongTensor()
#     all_precision = []
#     all_accuracy = []
    
#     model.eval()
#     with torch.no_grad():
#         for x, x_masks, rev_x, rev_x_masks, y, y_masks in test_loader:
#             n_eval = y.shape[0] - 1 if n == -1 else n
#             y_hat = model(x, x_masks, rev_x, rev_x_masks)
#             y_hat = F.softmax(y_hat, dim=-1)
#             y_multihot = indices_to_multihot(y, y_masks, y_hat)
#             k_correct = 0
#             total_precision = 0
#             total_accuracy = 0
#             for i in range(n_eval):
#                 visit_correct = 0
#                 y_true = y[i, y_masks[i]]
#                 _, y_pred = torch.topk(y_hat[i], k)

#                 for v in y_true:
#                     if v in y_pred:
#                         visit_correct += 1
                        
#                 visit_precision = visit_correct / min(k, len(y_true))
#                 visit_accuracy = visit_correct / len(y_true)
#                 k_correct += visit_correct
#                 total_precision += visit_precision
#                 total_accuracy += visit_accuracy

#             precision_k = total_precision / n_eval
#             accuracy_k = total_accuracy / n_eval
#             all_precision.append(precision_k)
#             all_accuracy.append(accuracy_k)
            
#     total_precision_k = np.mean(all_precision)
#     total_accuracy_k = np.mean(all_accuracy)
#     return total_precision_k, total_accuracy_k

    y_pred = torch.LongTensor()
    y_true = torch.LongTensor()
    all_precision = []
    all_accuracy = []
    
    model.eval()
    with torch.no_grad():
        for x, x_masks, rev_x, rev_x_masks, y, y_masks in test_loader:
            n_eval = y.shape[0] - 1 if n == -1 else n
            y_hat = model(x, x_masks, rev_x, rev_x_masks)
            y_hat = F.softmax(y_hat, dim=-1)
            #y_multihot = indices_to_multihot(y, y_masks, y_hat)
            nz_rows, nz_cols = torch.nonzero(y, as_tuple=True)
            k_correct = 0
            total_precision = 0
            total_accuracy = 0
            for i in range(n_eval):
                visit_correct = 0
                #y_true = y[i, y_masks[i]]
                y_true = nz_cols[nz_rows == i]
                _, y_pred = torch.topk(y_hat[i], k)
                for v in y_true:
                    if v in y_pred:
                        visit_correct += 1
                visit_precision = visit_correct / min(k, len(y_true))
                visit_accuracy = visit_correct / len(y_true)
                k_correct += visit_correct
                total_precision += visit_precision
                total_accuracy += visit_accuracy
            precision_k = total_precision / n_eval
            accuracy_k = total_accuracy / n_eval
            all_precision.append(precision_k)
            all_accuracy.append(accuracy_k)
            
    total_precision_k = np.mean(all_precision)
    total_accuracy_k = np.mean(all_accuracy)
    return total_precision_k, total_accuracy_k

In [17]:
def indices_to_multihot(indices, masks, y_hat):
    multihot = torch.zeros_like(y_hat, dtype=torch.float)
    for idx, row in enumerate(indices):
        y_idx = row[masks[idx]].unique()
        multihot[idx] = F.one_hot(y_idx, y_hat.shape[1]).sum(0).float()
    return multihot

In [18]:
def print_cpu_usage():
    load = psutil.getloadavg()[2]
    cpu_usage = (load/os.cpu_count()) * 100
    ram = psutil.virtual_memory()[2]
    print(f"CPU: {cpu_usage:0.2f}")
    print(f"RAM %: {ram}")
    return cpu_usage, ram

In [19]:
n_epochs = 100
%time train(baseline_retain, train_loader, val_loader, n_epochs)

CPU: 11.51
RAM %: 60.9
CPU: 11.58
RAM %: 62.0
Epoch: 1 	 Training Loss: 4.580161
Epoch: 1 	 Validation precision@k5: 0.5718, accuracy@k5: 0.2898
Epoch: 1 	 Validation precision@k10: 0.5313, accuracy@k10: 0.4477
Epoch: 1 	 Validation precision@k15: 0.5735, accuracy@k15: 0.5547
Epoch: 1 	 Validation precision@k20: 0.6381, accuracy@k20: 0.6355
Epoch: 1 	 Validation precision@k25: 0.7035, accuracy@k25: 0.7033
Epoch: 1 	 Validation precision@k30: 0.7538, accuracy@k30: 0.7538
CPU: 11.73
RAM %: 62.0
Epoch: 2 	 Training Loss: 3.888857
Epoch: 2 	 Validation precision@k5: 0.6273, accuracy@k5: 0.3330
Epoch: 2 	 Validation precision@k10: 0.5911, accuracy@k10: 0.5038
Epoch: 2 	 Validation precision@k15: 0.6263, accuracy@k15: 0.6071
Epoch: 2 	 Validation precision@k20: 0.6881, accuracy@k20: 0.6853
Epoch: 2 	 Validation precision@k25: 0.7479, accuracy@k25: 0.7477
Epoch: 2 	 Validation precision@k30: 0.7899, accuracy@k30: 0.7899
CPU: 11.96
RAM %: 62.0
Epoch: 3 	 Training Loss: 3.709499
Epoch: 3 	 Vali

CPU: 14.93
RAM %: 62.8
Epoch: 19 	 Training Loss: 3.450114
Epoch: 19 	 Validation precision@k5: 0.6773, accuracy@k5: 0.3631
Epoch: 19 	 Validation precision@k10: 0.6312, accuracy@k10: 0.5397
Epoch: 19 	 Validation precision@k15: 0.6668, accuracy@k15: 0.6468
Epoch: 19 	 Validation precision@k20: 0.7213, accuracy@k20: 0.7185
Epoch: 19 	 Validation precision@k25: 0.7775, accuracy@k25: 0.7773
Epoch: 19 	 Validation precision@k30: 0.8208, accuracy@k30: 0.8208
CPU: 15.08
RAM %: 62.8
Epoch: 20 	 Training Loss: 3.448546
Epoch: 20 	 Validation precision@k5: 0.6807, accuracy@k5: 0.3641
Epoch: 20 	 Validation precision@k10: 0.6323, accuracy@k10: 0.5405
Epoch: 20 	 Validation precision@k15: 0.6669, accuracy@k15: 0.6470
Epoch: 20 	 Validation precision@k20: 0.7225, accuracy@k20: 0.7197
Epoch: 20 	 Validation precision@k25: 0.7789, accuracy@k25: 0.7787
Epoch: 20 	 Validation precision@k30: 0.8212, accuracy@k30: 0.8212
CPU: 15.38
RAM %: 62.8
Epoch: 21 	 Training Loss: 3.445582
Epoch: 21 	 Validation 

Epoch: 36 	 Validation precision@k30: 0.8331, accuracy@k30: 0.8331
CPU: 18.23
RAM %: 62.8
Epoch: 37 	 Training Loss: 3.408657
Epoch: 37 	 Validation precision@k5: 0.6978, accuracy@k5: 0.3728
Epoch: 37 	 Validation precision@k10: 0.6573, accuracy@k10: 0.5598
Epoch: 37 	 Validation precision@k15: 0.6840, accuracy@k15: 0.6632
Epoch: 37 	 Validation precision@k20: 0.7419, accuracy@k20: 0.7389
Epoch: 37 	 Validation precision@k25: 0.7959, accuracy@k25: 0.7957
Epoch: 37 	 Validation precision@k30: 0.8362, accuracy@k30: 0.8362
CPU: 18.45
RAM %: 62.8
Epoch: 38 	 Training Loss: 3.406201
Epoch: 38 	 Validation precision@k5: 0.7033, accuracy@k5: 0.3756
Epoch: 38 	 Validation precision@k10: 0.6595, accuracy@k10: 0.5624
Epoch: 38 	 Validation precision@k15: 0.6899, accuracy@k15: 0.6688
Epoch: 38 	 Validation precision@k20: 0.7431, accuracy@k20: 0.7401
Epoch: 38 	 Validation precision@k25: 0.7964, accuracy@k25: 0.7962
Epoch: 38 	 Validation precision@k30: 0.8370, accuracy@k30: 0.8370
CPU: 18.63
RAM 

Epoch: 54 	 Validation precision@k25: 0.7997, accuracy@k25: 0.7995
Epoch: 54 	 Validation precision@k30: 0.8389, accuracy@k30: 0.8389
CPU: 21.63
RAM %: 62.7
Epoch: 55 	 Training Loss: 3.389363
Epoch: 55 	 Validation precision@k5: 0.7104, accuracy@k5: 0.3794
Epoch: 55 	 Validation precision@k10: 0.6669, accuracy@k10: 0.5679
Epoch: 55 	 Validation precision@k15: 0.6907, accuracy@k15: 0.6693
Epoch: 55 	 Validation precision@k20: 0.7468, accuracy@k20: 0.7437
Epoch: 55 	 Validation precision@k25: 0.7990, accuracy@k25: 0.7988
Epoch: 55 	 Validation precision@k30: 0.8371, accuracy@k30: 0.8371
CPU: 21.74
RAM %: 62.7
Epoch: 56 	 Training Loss: 3.388649
Epoch: 56 	 Validation precision@k5: 0.7061, accuracy@k5: 0.3775
Epoch: 56 	 Validation precision@k10: 0.6622, accuracy@k10: 0.5639
Epoch: 56 	 Validation precision@k15: 0.6906, accuracy@k15: 0.6692
Epoch: 56 	 Validation precision@k20: 0.7452, accuracy@k20: 0.7421
Epoch: 56 	 Validation precision@k25: 0.7991, accuracy@k25: 0.7989
Epoch: 56 	 Val

Epoch: 72 	 Validation precision@k20: 0.7468, accuracy@k20: 0.7436
Epoch: 72 	 Validation precision@k25: 0.7977, accuracy@k25: 0.7976
Epoch: 72 	 Validation precision@k30: 0.8365, accuracy@k30: 0.8365
CPU: 24.15
RAM %: 62.8
Epoch: 73 	 Training Loss: 3.384805
Epoch: 73 	 Validation precision@k5: 0.7091, accuracy@k5: 0.3794
Epoch: 73 	 Validation precision@k10: 0.6641, accuracy@k10: 0.5655
Epoch: 73 	 Validation precision@k15: 0.6936, accuracy@k15: 0.6719
Epoch: 73 	 Validation precision@k20: 0.7474, accuracy@k20: 0.7442
Epoch: 73 	 Validation precision@k25: 0.7970, accuracy@k25: 0.7969
Epoch: 73 	 Validation precision@k30: 0.8353, accuracy@k30: 0.8353
CPU: 24.27
RAM %: 62.8
Epoch: 74 	 Training Loss: 3.383311
Epoch: 74 	 Validation precision@k5: 0.7097, accuracy@k5: 0.3789
Epoch: 74 	 Validation precision@k10: 0.6631, accuracy@k10: 0.5646
Epoch: 74 	 Validation precision@k15: 0.6928, accuracy@k15: 0.6711
Epoch: 74 	 Validation precision@k20: 0.7456, accuracy@k20: 0.7425
Epoch: 74 	 Val

Epoch: 90 	 Validation precision@k15: 0.6941, accuracy@k15: 0.6724
Epoch: 90 	 Validation precision@k20: 0.7461, accuracy@k20: 0.7429
Epoch: 90 	 Validation precision@k25: 0.7988, accuracy@k25: 0.7986
Epoch: 90 	 Validation precision@k30: 0.8362, accuracy@k30: 0.8362
CPU: 26.82
RAM %: 62.8
Epoch: 91 	 Training Loss: 3.381402
Epoch: 91 	 Validation precision@k5: 0.7114, accuracy@k5: 0.3806
Epoch: 91 	 Validation precision@k10: 0.6644, accuracy@k10: 0.5655
Epoch: 91 	 Validation precision@k15: 0.6954, accuracy@k15: 0.6738
Epoch: 91 	 Validation precision@k20: 0.7451, accuracy@k20: 0.7420
Epoch: 91 	 Validation precision@k25: 0.7975, accuracy@k25: 0.7974
Epoch: 91 	 Validation precision@k30: 0.8364, accuracy@k30: 0.8364
CPU: 26.79
RAM %: 62.7
Epoch: 92 	 Training Loss: 3.382241
Epoch: 92 	 Validation precision@k5: 0.7140, accuracy@k5: 0.3814
Epoch: 92 	 Validation precision@k10: 0.6646, accuracy@k10: 0.5661
Epoch: 92 	 Validation precision@k15: 0.6929, accuracy@k15: 0.6712
Epoch: 92 	 Val

In [28]:
for k in range(5, 31, 5):
    precision_k, accuracy_k = eval_model(baseline_retain, test_loader, k=k)
    print(f'Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')

Validation precision@k5: 0.6537, accuracy@k5: 0.3494
Validation precision@k10: 0.6338, accuracy@k10: 0.5404
Validation precision@k15: 0.6770, accuracy@k15: 0.6569
Validation precision@k20: 0.7373, accuracy@k20: 0.7340
Validation precision@k25: 0.7941, accuracy@k25: 0.7939
Validation precision@k30: 0.8369, accuracy@k30: 0.8369


In [ ]:
torch.save(baseline_retain, os.path.join(CHECKPOINT_PATH, "BaselineRETAIN_100.pth"))